https://schoolforengineering.com/tutorial/huggingface-transformers-nlp-model-own-dataset/


Summarization with T5 colab notebook

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install torch
!pip install accelerate
!pip install datasets evaluate transformers rouge-score nltk

# Conectamos a HuggingFace

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
!pip install git-lfs

In [4]:
import pandas as pd

# Conectamos el entorno de Colab
from google.colab import drive
drive.mount('/content/drive')


# Ruta del archivo en Google Drive
#dataset_path = '/content/drive/My Drive/Profesional/7mo Semestre - Profesional/Bloques 2 y 3/Reto - Generative IA/T5/Dataset_Coca_Cola_Flan_T5.json'

#Otra cuenta, cargamos la carpeta GenText2Text a Mi unidad con un acceso directo
dataset_path = '/content/drive/My Drive/GenText2Text/T5/Dataset_Coca_Cola_Flan_T5.json'
df = pd.read_json(dataset_path)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,prompt,target
0,Generate a marketing offer:,Buy 4 cokes and get the last one for free!
1,Create a special deal for Coca Cola:,Get a 6-pack of Coca Cola for the price of 5!
2,Come up with a promotion for Coca Cola products:,Buy any two Coca Cola drinks and get 20% off t...
3,Design an offer for Coke lovers:,Purchase 3 cokes and receive a complimentary C...
4,Generate a discount for Coca Cola beverages:,Enjoy a special price of 50 pesos for a pack o...


In [5]:
# Calculamos el tamaño máximo de prompts
p = []
for i in range(0,len(df['prompt'])):
  p.append(len(df['prompt'][i]))
print(f'El tamaño máximo de prompts es: {max(p)}')


# Calculamos el tamaño máximo de targets
t = []
for i in range(0,len(df['target'])):
  t.append(len(df['target'][i]))
print(f'El tamaño máximo de prompts es: {max(t)}')

El tamaño máximo de prompts es: 84
El tamaño máximo de prompts es: 131


In [6]:
from sklearn.model_selection import train_test_split

# Dividir en entrenamiento (80%) y temporal (20%)
temp_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

# Dividir temporal en entrenamiento (75%) y validación (25%)
train_df, val_df = train_test_split(temp_df, test_size=0.25, shuffle=True, random_state=42)

# Imprimir tamaños de los conjuntos resultantes
print("Tamaño del conjunto de entrenamiento:", len(train_df))
print("Tamaño del conjunto de validación:", len(val_df))
print("Tamaño del conjunto de prueba:", len(test_df))


Tamaño del conjunto de entrenamiento: 602
Tamaño del conjunto de validación: 201
Tamaño del conjunto de prueba: 201


In [7]:
train_df.head()

,prompt,target
238,Come up with a vitamin boost offer for Fanta f...,Buy 3 Fantas and receive a discount on vitamin...
190,Devise a surprise deal for Coke lovers:,Buy any Coca Cola product and find a surprise ...
263,Think of a Fuze Tea and healthy living promotion:,Purchase a Fuze Tea and receive a discount on ...
758,Create a bundle deal for Sprite and Monster en...,Purchase a Sprite and a Monster together for a...
583,Come up with a special deal for Coca Cola and ...,Purchase a Coca Cola and a Del Valle juice tog...


In [8]:
train_df.dtypes

prompt    object
target    object
dtype: object

In [9]:
from datasets import Dataset
from evaluate import load

# Cargamos la métrica para evaluar
metric = load("rouge")

#Pasamos a formato de diccionario
train_dic = train_df.to_dict(orient='list')
val_dic = val_df.to_dict(orient='list')
test_dic = test_df.to_dict(orient='list')

# Creamos el dataset
ds_train = Dataset.from_dict(train_dic)
ds_val = Dataset.from_dict(val_dic)
ds_test = Dataset.from_dict(test_dic)


In [10]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [11]:
from datasets.dataset_dict import DatasetDict

# construct a DatasetDict from the datasets
datasetdict = DatasetDict({"train":ds_train,
                             "validation": ds_val,
                             "test":ds_test})
print(datasetdict)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'target'],
        num_rows: 602
    })
    validation: Dataset({
        features: ['prompt', 'target'],
        num_rows: 201
    })
    test: Dataset({
        features: ['prompt', 'target'],
        num_rows: 201
    })
})


# Tokenización del objeto DatasetDict

In [12]:
datasetdict['validation'][0]['target']

'Buy a Coca Cola DIY project kit and get a free tutorial for a creative project.'

In [13]:
# Divido
data_1 = datasetdict['validation']

In [14]:
data_1['prompt'][0]

'Generate a DIY project offer for Coca Cola enthusiasts:'

In [15]:
from transformers import T5Tokenizer
import torch

#Cargamos tokenizador de T5
model_checkpoint = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

#Tokenizamos una ejemplo del subconjunto 'train' con el parámetro de truncation
# tokenizer(datasetdict['train'][0]['target'], truncation=True, padding=False)

#Tokenizamos el subconjunto
t = tokenizer(datasetdict['train']['prompt'], truncation=True, padding=False)

#Creamos una función de tokenización
def tokenization(example):
  return tokenizer(example['prompt'])

#Aplicamos la función map con la función tokenization para que se tokenice el texto en batches
# tokenized_dataset = data_1.map(tokenization, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
max_input_length = 80  # Nos basamos en el vlor máximo de p y t
max_target_length = 130
prefix = 'Generate promotion:'

# Función para tokenización con truncation
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["prompt"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["target"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
datasetdict['train'][:2]

{'prompt': ['Come up with a vitamin boost offer for Fanta fans:',
  'Devise a surprise deal for Coke lovers:'],
 'target': ['Buy 3 Fantas and receive a discount on vitamin supplements from AdeS.',
  'Buy any Coca Cola product and find a surprise coupon at the bottom of the bag – discounts, freebies, and more!']}

In [18]:
preprocess_function(datasetdict['train'][:2])

{'input_ids': [[6939, 2206, 5546, 10, 5890, 15, 95, 28, 3, 9, 7869, 4888, 462, 21, 8362, 17, 9, 2675, 10, 1], [6939, 2206, 5546, 10, 2962, 3466, 15, 3, 9, 4158, 1154, 21, 638, 1050, 12891, 10, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[4783, 220, 8362, 17, 9, 7, 11, 911, 3, 9, 3898, 30, 7869, 11717, 45, 1980, 15, 134, 5, 1], [4783, 136, 25417, 3043, 9, 556, 11, 253, 3, 9, 4158, 8133, 44, 8, 2007, 13, 8, 2182, 3, 104, 9228, 6, 339, 4232, 7, 6, 11, 72, 55, 1]]}

Utilizamos el método map en el objeto dataset que contiene los splits de datos para aplicar la función de preprocesamiento.

In [19]:
tokenized_datasets = datasetdict.map(preprocess_function, batched=True)

Map:   0%|          | 0/602 [00:00<?, ? examples/s]

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['prompt', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 602
    })
    validation: Dataset({
        features: ['prompt', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 201
    })
    test: Dataset({
        features: ['prompt', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 201
    })
})

In [21]:
from datasets import load_dataset
t = load_dataset('LeoR2/T5-small-Coca', split='train')

# Fine-tuning

La tarea que queremos hacer es una tarea de **sequence-to-sequence** o *seq2seq* en donde el input al modelo es una secuencia de texto y el output es otra. Por ello usaremos el approach correspondiente para ajustar finamente el modelo.

In [22]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [23]:
batch_size = 6
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-Coca",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

Utilizamos un Collator para hacer padding a los inputs y targets correspondientes para que tengan la misma longitud en cada batch.

In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [25]:
import nltk
import numpy as np
# Descargar el recurso 'punkt'
nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Finally we apply the fine tune to the model by using the *train* method

In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,nan,35.841900,14.138700,29.684600,29.769800,17.084600
2,No log,nan,35.841900,14.138700,29.684600,29.769800,17.084600


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=202, training_loss=0.0, metrics={'train_runtime': 67.8452, 'train_samples_per_second': 17.746, 'train_steps_per_second': 2.977, 'total_flos': 10364121772032.0, 'train_loss': 0.0, 'epoch': 2.0})

In [28]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1700183763.9a95aa27ca08.12896.0:   0%|          | 0.00/6.67k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

'https://huggingface.co/LeoR2/flan-t5-small-finetuned-Coca/tree/main/'

# Verificación de estructura de datos existentes en la Hub de HuggingFace

In [29]:
from datasets import load_dataset
squad = load_dataset('rotten_tomatoes', split='train')
print(squad)

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})


In [30]:
raw_dataset = load_dataset('xsum')
print(raw_dataset)

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})


In [31]:
squad['validation'].to_pandas()
squad['validation'][0]

KeyError: ignored

# Creación de nuestro dataset